In [1]:
import pandas as pd
import math
import yfinance as yf
import plotly.graph_objects as go
from plotly.subplots import make_subplots

# ativos
tickers = ["DIRR3", "CURY3", "JHSF3",
           "MRVE3", "EVEN3",
           "TRIS3", "EZTC3",
           "TEND3", "CYRE3",
           "HBOR3", "GFSA3", "TCSA3",
           "LAVV3", "PLPL3", "MTRE3", "MDNE3" 
            ]
period = "10y"
interval = "1wk"

In [2]:
data_dict = {}  

for ticker in tickers:
    df = yf.download(ticker+".SA", period=period, interval=interval, auto_adjust=True, progress=False)[['Close']]
    data_dict[ticker] = df  

In [3]:
# grafico
height=800
width=800

mma = 15

fig = make_subplots(
    rows=math.ceil(len(tickers)/2), cols=2, shared_xaxes=False, 
    subplot_titles= tickers , 
    vertical_spacing=0.1
)

for i, ticker in enumerate(tickers):
    df = data_dict[ticker][('Close', ticker+".SA")]  
    avg = df.rolling(window=mma, min_periods=1).mean()
    
    row = (i // 2) + 1
    col = (i % 2) + 1

    fig.add_trace(go.Scatter(
        x=df.index, y=avg, mode='lines',
        line=dict(color='blue', width=1),
        name="Média", showlegend=False), row=row, col=col)

fig.update_layout(title_text=f"<b>Construção civil</b><br><sub>mm{mma} do fechamento semanal</sub>",
                  height= height, width=width)
fig.update_xaxes(rangeslider_visible=False,
    tickformat="%Y",  # Mostrar apenas o ano
    dtick="M36",  # Intervalo de meses entre os ticks
    ticklabelmode="period"  # Coloca o rótulo no início do período)
)
fig.show()


In [69]:
import requests
from io import StringIO

#pra fingir que é um browser
header = {
  "User-Agent": "Mozilla/5.0 (X11; Linux x86_64) AppleWebKit/537.36 (KHTML, like Gecko) Chrome/50.0.2661.75 Safari/537.36",
  "X-Requested-With": "XMLHttpRequest"
}
url = 'https://www.fundamentus.com.br/resultado.php'
#junta com a requests
r = requests.get(url, headers=header)
# read_html do pandas põe a tabela num dataframe
funds = pd.read_html(StringIO(r.text), decimal=',', thousands='.')[0]
dfunds = pd.DataFrame(funds)

media_pl = dfunds[dfunds['Papel'].isin([ticker.removesuffix('.SA') for ticker in tickers])]['P/L'].mean()
print(f'\nO P/L médio dessas empresas é {media_pl:.2f}')

print(dfunds[dfunds['Papel'].isin([ticker.removesuffix('.SA') for ticker in tickers])]['P/L'].describe())

ibra = pd.read_csv('data/Cart_IBrA.csv', 
                 sep=';',
                 encoding='latin1',
                 engine='python',
                 skiprows=2,    
                 skipfooter=2,
                 usecols=[0],header=None)[0].to_list()
intersecao = sorted(set(tickers) & set(ibra))

media_pl_indice = dfunds[dfunds['Papel'].isin(intersecao)]['P/L'].mean()
print(f'\nO P/L médio dessas empresas que estão no índice amplo ({intersecao}) é {media_pl_indice:.2f}')



O P/L médio dessas empresas é 6.57
count    16.000000
mean      6.568125
std       4.532174
min      -3.450000
25%       6.990000
50%       8.080000
75%       8.617500
max      12.700000
Name: P/L, dtype: float64

O P/L médio dessas empresas que estão no índice amplo (['CURY3', 'CYRE3', 'DIRR3', 'EVEN3', 'EZTC3', 'GFSA3', 'HBOR3', 'JHSF3', 'LAVV3', 'MDNE3', 'MRVE3', 'PLPL3', 'TEND3']) é 6.90
